In [2]:
import os
from ensure import ensure_annotations

In [3]:
os.chdir("../")

In [4]:
!pwd

/Users/ngkuissi/Dev/Image_Search_Engine


In [5]:
from dataclasses import dataclass
from pathlib import Path
from typing import List

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    model_path: Path
    feature_dir: Path
    image_path_list_dir: Path
    n_jobs: int
    n_neighbors: int

In [6]:
from imageSearchEngine.constants import *
from imageSearchEngine.utils.file_helpers import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    @ensure_annotations
    def __init__(
        self,
        config_filepath:Path = CONFIG_FILE_PATH,
        params_filepath:Path = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    @ensure_annotations
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.model_trainer
        create_directories([config.root_dir])
        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            model_path=config.model_path,
            n_jobs=params.n_jobs,
            n_neighbors=params.n_neighbors,
            feature_dir = config.feature_dir,
            image_path_list_dir= config.image_path_list_dir

        )

        return model_trainer_config


In [8]:
import os
from tqdm import tqdm
from pathlib import Path
import pickle
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from imageSearchEngine.logging.logger import log
from imageSearchEngine.exception import CustomException


In [29]:
class ModelTrainer:

    def __init__(self, config:ModelTrainerConfig):
        self.config = config
    
    def get_image_class(self) -> List:
        path_list = pickle.load(open(self.config.image_path_list_dir,'rb'))
        image_class = []
        for image_path in tqdm(path_list):
            class_name = image_path.split('/')[-3]
            image_class.append(class_name)
        log.info(f"retrived all the image classes from the path: {self.config.image_path_list_dir}")
        return image_class
    
    def train_model(self):
        image_labels = self.get_image_class()
        image_feature = pickle.load(open(self.config.feature_dir,'rb'))
        log.info(f"retrived all the features images that were produced earlier in the path: {self.config.feature_dir}")
        X = np.array(image_feature)
        y = np.array(image_labels)
        knn_classifier = KNeighborsClassifier(n_jobs=-1, n_neighbors=self.config.n_neighbors)
        knn_classifier = knn_classifier.fit(X, y)
        log.info("the model was fully trained")
        
        with open(self.config.model_path, 'wb') as file:
            pickle.dump(knn_classifier, file) 
        log.info(f"this model was saved to the path, {self.config.model_path}")
        
    

In [30]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config= model_trainer_config)
    model_trainer.train_model()
except Exception as e:
    raise CustomException(e)

[2023-10-06 17:52:02,656: INFO: file_helpers: yaml file: config/config.yaml loaded successfully]
[2023-10-06 17:52:02,658: INFO: file_helpers: yaml file: params.yaml loaded successfully]
[2023-10-06 17:52:02,658: INFO: file_helpers: created directory at: artifacts]
[2023-10-06 17:52:02,659: INFO: file_helpers: created directory at: artifacts/model]


100%|██████████| 100000/100000 [00:00<00:00, 3827862.70it/s]

[2023-10-06 17:52:02,693: INFO: 1898211959: retrived all the image classes from the path: artifacts/embeddings/path_list.pkl]


[2023-10-06 17:52:02,976: INFO: 1898211959: retrived all the features images that were produced earlier in the path: artifacts/embeddings/embedding.pkl]
['n02795169' 'n02795169' 'n02795169' 'n02795169' 'n02795169' 'n02795169'
 'n02795169' 'n02795169' 'n02795169' 'n02795169' 'n02795169' 'n02795169'
 'n02795169' 'n02795169' 'n02795169' 'n02795169' 'n02795169' 'n02795169'
 'n02795169' 'n02795169' 'n02795169' 'n02795169' 'n02795169' 'n02795169'
 'n02795169' 'n02795169' 'n02795169' 'n02795169' 'n02795169' 'n02795169'
 'n02795169' 'n02795169' 'n02795169' 'n02795169' 'n02795169' 'n02795169'
 'n02795169' 'n02795169' 'n02795169' 'n02795169' 'n02795169' 'n02795169'
 'n02795169' 'n02795169' 'n02795169' 'n02795169' 'n02795169' 'n02795169'
 'n02795169' 'n02795169' 'n02795169' 'n02795169' 'n02795169' 'n02795169'
 'n02795169' 'n02795169' 'n02795169' 'n02795169' 'n02795169' 'n02795169'
 'n02795169' 'n02795169' 'n02795169' 'n02795169' 'n02795169' 'n02795169'
 'n02795169' 'n02795169' 'n02795169' 'n02795

In [17]:
!pwd
!ls

/Users/ngkuissi/Dev/Image_Search_Engine/research


01_data_ingestion.ipynb       image_search_engine_toy.ipynb
02_data_clearning.ipynb       logs
03_feature_retrival.ipynb     test.ipynb
